## Displacement Random Walk Controversy

In [21]:
import networkx as nx
import pandas as pd
import numpy as np
import csv
import datetime as dt
import re
import os
import json
import random
import matplotlib.pyplot as plt
from networkx.algorithms.shortest_paths.generic import average_shortest_path_length
from networkx.algorithms.centrality import degree_centrality
import warnings
warnings.filterwarnings("ignore")

##### Read Graph and communities file

In [7]:
path= "C:/MyPath/Graphs/"

In [96]:
G = nx.read_weighted_edgelist(path + "vax_twitter_edgelist.txt",
                              delimiter=",")
f1 = open("C:/MyPath/vax_twittercommunity0.csv" )
f2 = open("C:/MyPath/vax_twittercommunity0.csv" )

##### Compute DRWC metric

In [97]:
lines1 = f1.readlines()
lines1= list(filter(('\n').__ne__, lines1 ))
dict_left = {}
for line in lines1:
        line = line.strip()
        dict_left[line] = 1
lines2 = f2.readlines()
lines2= list(filter(('\n').__ne__, lines2 ))
dict_right = {}

for line in lines2:
    line = line.strip()
    dict_right[line] = 1

In [98]:
part0= set(dict_left.keys())
part1= set(dict_right.keys())

In [99]:
cont = 0
for node in G.nodes(data=True):
    if node[0] in part0:
        G.nodes[node[0]]['com'] = 0
    else:
        G.nodes[node[0]]['com'] = 1
    G.nodes[node[0]]['id'] = cont
    cont += 1

In [147]:
def start_random_walks_side(node, graph, steps):
    max_restart = 20
    result = 0
    cont = 0
    start_node_com = node[1]['com']
    steps_done = 0
    
    change_times = 0
    
    #print(f'SELECTED NODE: {node[0]}')
    #print(f'STARTER NODE COMMUNITY: {start_node_com}')
    
    current_node = node[0]
    
    while steps_done <= steps:
        list_edges = list(graph.edges(current_node))
        if len(list_edges) != 0:
            next_node = list_edges[random.randint(0,len(list_edges)-1)][1]
            if graph.nodes[next_node]["com"] != start_node_com:
                change_times += 1
            steps_done += 1
    result = 1 - (change_times/steps)
    #print(f'RISULTATO: {result}')
    #print(f'Restart: {max_restart}')
    return result

In [148]:
def change_side_controversy(start_graph, sample_size, num_steps, part0, part1):
     
    result = list()
    
    for_community_0 = int(len(part0)*sample_size)
    for_community_1 = int(len(part1)*sample_size)
    
    sample_com_0 = for_community_0
    sample_com_1 = for_community_1
    
    already_taken = list()
    
    graph= start_graph
    
    while True:
        node_id = random.randint(0, len(graph)-1)
        for node in graph.nodes(data=True):
            if node[1]['id'] == node_id:
                break
        if len(list(graph.edges(node[0]))) > 0 and node_id not in already_taken:
            already_taken.append(node_id)
            com = node[1]['com']
            if not((for_community_0 == 0 and com == 0) or (for_community_1 == 0 and com == 1)):
                
                result.append(start_random_walks_side(node, graph, num_steps))
                if com == 0:
                    for_community_0 -= 1
                else:
                    for_community_1 -= 1
                
                # print(for_community_0 + for_community_1)
                    
                if int(for_community_0 + for_community_1) <= 0:
                    break
                elif (for_community_0 + for_community_1) == int((sample_com_0 + sample_com_1)*0.75):
                    pass
                    
                elif (for_community_0 + for_community_1) == int((sample_com_0 + sample_com_1)*0.5):
                    pass
                   
                elif (for_community_0 + for_community_1) == int((sample_com_0 + sample_com_1)*0.25):
                    pass
                    
                    
    score = np.mean(result)
    print(f'Change side controversy for : {np.mean(score)}')
    return np.mean(round(score, 4))
    

In [149]:
shortest_path = round(average_shortest_path_length(G))
print(shortest_path)

3


In [ ]:
change_side_controversy(G, 0.6, shortest_path*2, part0, part1)

# Authorative Random Walk Controversy

In [43]:
def top_out_degree(graph, perc, com):
    
    cont_com = 0
    
    for node in graph.nodes(data=True):
        if node[1]['com'] == com:
            cont_com += 1
    
    num_nodes = int(cont_com * (perc/100))
    
    top_out_degree = degree_centrality(graph)
    top_out_degree_sorted = {k: v for k, v in sorted(top_out_degree.items(), key=lambda item: item[1], reverse=True)}
    
    top_user = list()
    for node in top_out_degree_sorted:
        if graph.nodes[node]['com'] == com:
            top_user.append(node)
            
        num_nodes -= 1
        
        if num_nodes == 0:
            break 
            
    return top_user

In [44]:
def start_random_walks(node, graph, num_steps, top_com_0, top_com_1):
    result = 0
    start_node_com = int(node[1]['com'])
    steps_done = 0
    

    current_node = node[0]
    
    while True:
        list_edges = list(graph.edges(current_node))
        if len(list_edges) != 0:
            next_node = list_edges[random.randint(0,len(list_edges)-1)][1]
            steps_done += 1
            #print(f'Next node: {next_node}')
            #print(f'Next node com: {graph.nodes[next_node]["com"]}')
            if next_node in top_com_0:
                if start_node_com == 0:
                    return 0
                else:
                    return 1
            elif next_node in top_com_1:
                if start_node_com == 0:
                    return 2
                else:
                    return 3
        if steps_done >= num_steps:
            return -4
    
    print(f'RISULTATO: {result}')
    print(f'Restart: {max_restart}')
    return result

In [45]:
def random_walks_centrality(graph, num_steps):
    
    
   
   
    top_com_0 = top_out_degree(graph, 25, 0)
    top_com_1 = top_out_degree(graph, 25, 1)
    
    start_0_end_0 = 0
    start_0_end_1 = 0
    
    start_1_end_1 = 0
    start_1_end_0 = 0
    
    for top_node in top_com_0:
        for node in graph.nodes(data=True):
            if node[0] == top_node:
                break 
        
        result = start_random_walks(node, graph, num_steps, top_com_0, top_com_1)
        while result == 4:
            result = start_random_walks(node, graph, num_steps, top_com_0, top_com_1)

        if result == 0:
            start_0_end_0 += 1
        elif result == 2:
            start_0_end_1 += 1
    
            
    for top_node in top_com_1:
        for node in graph.nodes(data=True):
            if node[0] == top_node:
                break 

        result = start_random_walks(node, graph, num_steps, top_com_0, top_com_1)

        while result == 4:
            result = start_random_walks(node, graph, num_steps, top_com_0, top_com_1)
        
        if result == 1:
            start_1_end_0 += 1
        elif result == 3:
            start_1_end_1 += 1

          
    #print(f'start_0_end_0: {start_0_end_0}')
    #print(f'start_1_end_0: {start_1_end_0}')
    #print(f'start_0_end_1: {start_0_end_1}')
    #print(f'start_1_end_1: {start_1_end_1}')
    #print()
    
    start_0_end_0 = start_0_end_0/(len(top_com_0))    
    start_1_end_0 = start_1_end_0/(len(top_com_1))   
    start_0_end_1 = start_0_end_1/(len(top_com_0))   
    start_1_end_1 = start_1_end_1/(len(top_com_1)) 
    #print()
                    
    #print(f'start_0_end_0: {start_0_end_0}')
    #print(f'start_1_end_0: {start_1_end_0}')
    #print(f'start_0_end_1: {start_0_end_1}')
    #print(f'start_1_end_1: {start_1_end_1}')    
    score = start_0_end_0*start_1_end_1-start_1_end_0*start_0_end_1
    #print(f'RandomWalk top degree score for {com_type}: {round(score, 4)}')
    #print()
    
    return round(score, 4)
    

In [ ]:
random_walks_centrality(G, 20) #change % of authorative nodes, in this example top 20%